Item 40 Initialize Parent Classes with super  

Things to Remember
- Python's standard method resolution order (MRO) solves the problems of superclass initialization order and diamond inheritance.
- Use the super built-in function with zero arguments to initialize parent classes.

In [ ]:
class MyBaseClass:
    def __init__(self, value):
        self.value = value

In [ ]:
# - call parent's  __init__ method directly
# - works fine in simple cases
class MyChildClass(MyBaseClass):
    def __init__(self):
        MyBaseClass.__init__(self, 5)

print(MyChildClass().value)


In [ ]:
class TimesTwo:
    def __init__(self):
        self.value *= 2
class PlusFive:
    def __init__(self):
        self.value += 5

In [ ]:
# unpredictable behavior in multiple inheritance
class OneWay(MyBaseClass, TimesTwo, PlusFive):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

foo = OneWay(5)
print('First ordering value is (5 * 2) + 5 =', foo.value)

# - your intension is to plus 5 first and then times 2
#  
class AnotherWay(MyBaseClass, PlusFive, TimesTwo):
    def __init__(self, value):
        # - the order of the calls to the 
        #   parent class constructors doesn't
        #   match the order of the parent 
        #   classes in its definition 
        #   (PlusFive then TimesTwo)
        # - bugs like this are difficult 
        #   to spot  
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)


bar = AnotherWay(5)
print('Second ordering value is', bar.value) # still 15 not 20


In [ ]:
# diamond inheritance
# - two separate classes that have the same superclass 
#   somewhere in the hierarchy
class TimesSeven(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value) # reset the value
        self.value *= 7

class PlusNine(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value) # reset the value
        self.value += 9

In [ ]:
class ThisWay(TimesSeven, PlusNine):
    def __init__(self, value):
        TimesSeven.__init__(self, value)
        # - completely ignore the effect
        #   of the TimesSeven constructor
        # - this behavior can be very 
        #   difficult to debug in more
        #   complex cases
        PlusNine.__init__(self, value)

foo = ThisWay(5)
print('Should be (5 * 7) + 9 = 44 but is', foo.value)

Solution to these problems? the super built-in function and standard method resolution order (MRO)
- super ensures that common superclasses in diamond hierarchies are run only once (see Item 48)
- The MRO defines the ordering in which superclasses are initialized, following an algorithm called C3 linearization

In [ ]:
class TimesSevenCorrect(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)
        self.value *= 7 
class PlusNineCorrect(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)
        self.value += 9

class GoodWay(TimesSevenCorrect, PlusNineCorrect):
    def __init__(self, value):
        # - calling super will make sure
        #   the top part of the diamond
        #   , MyBaseClass.__init__, is 
        #   run pnly a single time 
        super().__init__(value) 

foo = GoodWay(5)
print('Should be 7 * (5 + 9) = 98 and is', foo.value)



In [ ]:
# observe MRO
mro_str = '\n'.join(repr(cls) for cls in GoodWay.mro())
print(mro_str) # like call stack

what happens when you call GoodWay(5) ?
- the order of the calls is TimesSevenCorrect.\__init\__ -> PlusNineCorrect.\__init\__ -> MyBaseClass.\__init\__
- the excution order is in the opposite order (like nested function calls)
  - MyBaseClass.\__init\__ assigns the value to 5
  - PlusNineCorrect.\__init\__ changes the value to 14 ( 5 + 9 )
  - TimesSevenCorrect.\__init\__ changes the value to 98 ( 14 x 7 )

calling super with two parameters
- the first is the type of the class whose MRO parent view you are trying to access
- the second is the instance on which to access that view 


In [32]:
class ExplicitTrisect(MyBaseClass):
    def __init__(self, value):
        super(ExplicitTrisect, self).__init__(value)
        self.value /= 3

class AutomaticTrisect(MyBaseClass):
    def __init__(self, value):
        # -  __class__ is a reference to the
        #    type of the current instance
        super(__class__, self).__init__(value)
        self.value /= 3

class ImplicitTrisect(MyBaseClass):
    def __init__(self, value):
        # - let Python's compiler
        #   do the work to provide
        #   the correct parameters
        super().__init__(value)
        self.value /= 3

In [33]:
assert ExplicitTrisect(9).value == 3
assert AutomaticTrisect(9).value == 3
assert ImplicitTrisect(9).value == 3

Note
- The only time you should provide parameters to super is in situations where you need to access the specific functionality of a superclass's implementation from a child.